In [1]:
## from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd
import numpy as np
import scipy.optimize
import re
import os
import importlib

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter, FuncFormatter


import trmc.kin as kin
import trmc.load as load
import trmc.analysis as analysis
import trmc.plot as plot
from trmc.plot import exp_formatter

import xarray as xr

mpl.rcParams['figure.figsize'] = [8, 8]
mpl.rc('font',**{'size' : 18})

from IPython.display import clear_output

import itertools

from collections import OrderedDict

# Bi data (no dark cavity sweeps)

In [2]:
# basedir = '/home/lee/OneDrive/Data/TRMC/Refined/freqdep/'
basedir = 'C:\\Users\\aspit\\OneDrive\\Data\\TRMC\\Refined\\Freqdep\\'
fps = OrderedDict()

fps['A (0%)'] = basedir + 'Bi_A_2/'
fps['D (0.001%)'] = basedir + 'Bi_D_2/'
fps['C (0.01%)'] = basedir + 'Bi_C_2/'

da_sw = load.sweeps2ds(fps)['Vsignal(V)']

In [3]:
das = []
das_bv = []

for samp in fps:
    direc = os.path.join(fps[samp],'FreqFluence')
    print('Loading data for ' + samp)
    s_fps = load.freqfluence_flist(direc)
#     print(s_fps)
    s,backvs = load.freqfluence_load(s_fps)
    print('Converting to Dataset')
    da = xr.DataArray.from_series(s)
    da = da.expand_dims('sample')
    da['sample'] = [samp]
    das.append(da)
    
    da_bv = xr.DataArray.from_series(backvs)
    da_bv = da_bv.expand_dims('sample')
    da_bv['sample'] = [samp]
    das_bv.append(da_bv)
    
da_dv = xr.concat(das, dim = 'sample')
da_dv = da_dv*(-1) #voltage is flipped
da_bv = xr.concat(das_bv,dim = 'sample')
print('done')

Loading data for A (0%)
Converting to Dataset
Loading data for D (0.001%)
Converting to Dataset
Loading data for C (0.01%)
Converting to Dataset
done


In [4]:
da_dv_bi = da_dv
da_bv_bi = da_bv
da_sw_bi = da_sw

%store da_dv_bi
%store da_bv_bi
%store da_sw_bi

Stored 'da_dv_bi' (DataArray)
Stored 'da_bv_bi' (DataArray)
Stored 'da_sw_bi' (DataArray)


# BHJ (dark cavity sweep)

In [5]:
# basedir = '/home/lee/OneDrive/Data/TRMC/Refined/freqdep/'
basedir = 'C:\\Users\\aspit\\OneDrive\\Data\\TRMC\\Refined\\Freqdep\\'
fps = OrderedDict()

fps['bhj'] = basedir + 'PH3TUA/'

da_sw = load.sweeps2ds(fps)['Vsignal(V)']


In [6]:
das = []
das_bv = []

for samp in fps:
    direc = os.path.join(fps[samp],'FreqFluence')
    print('Loading data for ' + samp)
    s_fps = load.freqfluence_flist(direc)
#     print(s_fps)
    s,backvs = load.freqfluence_load(s_fps)
    print('Converting to Dataset')
    da = xr.DataArray.from_series(s)
    da = da.expand_dims('sample')
    da['sample'] = [samp]
    das.append(da)
    
    da_bv = xr.DataArray.from_series(backvs)
    da_bv = da_bv.expand_dims('sample')
    da_bv['sample'] = [samp]
    das_bv.append(da_bv)
    
da_dv = xr.concat(das, dim = 'sample')
da_dv = da_dv*(-1) #voltage is flipped
da_bv = xr.concat(das_bv,dim = 'sample')
print('done')

no match for file TestSweep_FluenceSweep.csv
Converting to Dataset
done


In [7]:
direc = os.path.join(fps['bhj'],'FreqFluence')
fpsdcs = load.freqdcs_flist(direc)
das = []

for direction,freqt in fpsdcs.index:
    fp = fpsdcs.loc[direction,freqt]
    s = load.loadsweep(fp,defaultV = 0.025)
    s = s.rename(s.name.replace(' ', ''))
    s.index = s.index.rename('freq')
    da = xr.DataArray.from_series(s)
    da = da.assign_coords(direction = direction).expand_dims('direction')
    da = da.assign_coords(freqt = freqt).expand_dims('freqt')
    das.append(da)
    
da_dcs = xr.merge(das)['Vsignal(V)']

In [8]:
da_dv_bhj = da_dv
da_bv_bhj = da_bv
da_sw_bhj = da_sw
da_dcs_bhj = da_dcs

%store da_dv_bhj
%store da_bv_bhj
%store da_sw_bhj
%store da_dcs_bhj 

Stored 'da_dv_bhj' (DataArray)
Stored 'da_bv_bhj' (DataArray)
Stored 'da_sw_bhj' (DataArray)
Stored 'da_dcs_bhj' (DataArray)
